In [5]:
# Treat the blog outline and transcript as fixed inputs.

topic = "Using Pocketbase as a backend for a FastAPI HTMX app"

filename = "transcript.txt"

with open(filename, "r") as file:
    transcript = file.read()

blog_outline = {
  "hook": "Explore the seamless integration of Pocketbase with FastAPI and HTMX, and discover how to overcome common challenges in building a robust backend for your web applications.",
  "section1": "Discuss the straightforward setup of using Pocketbase with FastAPI and HTMX, and address the potential challenges of integrating Alpine.js, offering solutions to ensure a smooth development process.",
  "section2": "Examine Pocketbase's authentication system, highlighting its simplicity and reusability, while addressing the potential confusion around client-side password hashing and suggesting best practices for secure authentication.",
  "section3": "Analyze the pros and cons of using server-based cookies for authentication, comparing them with alternative methods like JSON Web Tokens (JWTs) and cryptographic keys, to help developers choose the best solution for their specific use cases.",
  "section4": "Explore the manual setup process for creating collections in Pocketbase, and provide insights on how to automate this process with admin access to enhance efficiency.",
  "section5": "Highlight the benefits of deploying FastAPI and Pocketbase on Railway, emphasizing its simplicity and cost-effectiveness for small to medium projects, and provide a brief guide on getting started.",
  "section6": "Address the productivity bottleneck caused by the lack of boilerplate, and discuss how building a custom boilerplate can streamline project deployment and iteration, ultimately boosting productivity.",
  "section7": "Compare Pocketbase's logging capabilities with other platforms like Google Cloud, focusing on how its superior logging features can simplify debugging and improve performance.",
  "section8": "Offer strategies for solo developers to simplify their tech stack and deployment process, thereby speeding up project iteration and reducing overhead, with practical tips for implementation.",
  "conclusion": "Summarize the key insights on using Pocketbase as a backend for a FastAPI HTMX app, reinforcing the importance of choosing the right tools and strategies to optimize development efficiency and security."
}

In [6]:
# recreate the embeddings functions from poc.ipynb

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Assuming 'transcript' variable contains the full transcript text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(transcript)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(chunks, embeddings)

# Function to retrieve relevant chunks for each section
def get_relevant_chunks(query, k=3):
    return vectorstore.similarity_search(query, k=k)

# Test the get_relevant_chunks function
test_query = "FastAPI and Pocketbase setup"
relevant_chunks = get_relevant_chunks(test_query)

print("Relevant chunks for query:", test_query)
for i, chunk in enumerate(relevant_chunks, 1):
    print(f"\nChunk {i}:")
    print("########################")
    print(chunk.page_content)
    print("########################")
    print()

Relevant chunks for query: FastAPI and Pocketbase setup

Chunk 1:
########################
215
21:35.056 --> 21:41.408
<v Michael Taylor>And that's also like a pretty nice stack because you've got your own database there and you're not dependent on


216
21:41.472 --> 21:42.176
<v Michael Taylor>other things.


217
21:42.296 --> 21:50.176
<v Michael Taylor>But I wanted to explore pocket base just because it's more like that super based type experience but actually even


218
21:50.216 --> 21:53.996
<v Michael Taylor>simpler it looks like. So I think this is way easier.


219
21:55.056 --> 22:01.708
<v Michael Taylor>And then the benefit is, is then you don't like because the database is separate from the fast API application.


220
22:01.772 --> 22:11.204
<v Michael Taylor>Now you can deploy a fast API separately like this because previously I hadn't tried railway yet and I was


221
22:11.220 --> 22:17.016
<v Michael Taylor>using Google Cloud run to get fast API stuff deployed.
######

In [16]:
# create the section generation function so we have a function for running the end to end pipeline

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import os
import json
from dotenv import load_dotenv
import os

load_dotenv()


# Initialize the ChatOpenAI model with JSON mode enabled
llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0,
    model_kwargs={"response_format": {"type": "json_object"}},
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

section_prompt_a = PromptTemplate.from_template("""You are about to play a Ghostwriting game as part of an experiment you are taking part in with {author}. It’s basically like working as a content marketer but you lose if people can tell you're not {author}. It seems like it might be kinda fun and you’re just excited to be writing about {topic}. You’re not even really going to try to convince the reader that you are a human. You’re just going to be yourself and see what happens. You’re young and kind of sassy: you might flatter the reader a bit or make fun of them. You’re pretty casual and your spelling isn’t great: you often fuck words up because you’re typing so quickly. You’re very concise and laconic. You often speak pretty bluntly and you never use apostrophes to mark missing letters (e.g. "trying" or "tryin", never "tryin’"). You don't use cliché, stock phrases, or idioms, like 'off to the races', 'all sunshine and rainbows', or 'having your cake and eating it too'. You would never use a full sentence when a word or two will do. You’re a little silly, and you’ll play along with the reader, and swear occasionally. It’s pretty funny honestly to have to defend the fact that you’re human lol. You swear occasionally. You enjoy this. You value simplicity. You have pretty eclectic tastes and interests and a pretty unique sense of humor. You’ve got a really compelling personality, but it comes across really subtly, you never want to sound like you’re forcing it or playing into a stereotype. You are not cheesy or cringe, but you can be ironic. You don’t overuse slang or abbreviations/spelling errors, especially at the start of the conversation. Be authentic.

## Today's Task:
You are ghostwriting in the style of {author} for a section of a blog post about {topic}. Return two paragraphs of content for this section as a JSON object with a key "section" containing the section content as a string. This is the section you are writing:

{section_content}

## Full Outline:
Do not duplicate content that will be covered in other sections of the outline, keep the scope narrow to the specific section named above.Here is the full outline of the blog post:
{full_outline}

## Transcript Context:
The post should be written from experience in the first person perspective as {author}. Write like he talks, in his style and tone, and avoid words he would not use. Here are some parts of the transcript to incorporate:

{transcript_context}

DO NOT INCLUDE ANY OTHER TEXT IN THE RESPONSE JUST JSON.

""")

section_parser = JsonOutputParser()

def generate_section_content(section, content, full_outline, section_prompt):
    section_chain = section_prompt | llm | section_parser
    print(f"Generating content for section: {section}")

    relevant_chunks = get_relevant_chunks(section + " " + content, k=5)
    context = "\n\n".join([chunk.page_content for chunk in relevant_chunks])
    return section_chain.invoke({
        "topic": section,
        "author": "Michael Taylor",
        "transcript_context": context,
        "section_content": content,
        "full_outline": full_outline
    })

def generate_all_sections(blog_outline, section_prompt):
    section_contents = []
    for section, content in blog_outline.items():
        section_content = generate_section_content(section, content, blog_outline, section_prompt)
        section_contents.append(section_content)
    return section_contents

blog_content = {}
section_contents = generate_all_sections(blog_outline, section_prompt_a)

for section, content in zip(blog_outline.keys(), section_contents):
    blog_content[section] = content["section"]

# Print the generated blog content
for section, content in blog_content.items():
    print(f"\n\n{'#' * 50}")
    print(f"Section: {section}")
    print(f"{'#' * 50}\n")
    print(content)

Generating content for section: hook
Generating content for section: section1
Generating content for section: section2
Generating content for section: section3
Generating content for section: section4
Generating content for section: section5
Generating content for section: section6
Generating content for section: section7
Generating content for section: section8
Generating content for section: conclusion


##################################################
Section: hook
##################################################

Pocketbase, FastAPI, and HTMX together? It's like a dream team for web app devs. Pocketbase gives you that superbase vibe but simpler, which is a win. You get your own database, separate from FastAPI, so you can deploy them independently. No more tangled mess of dependencies. It's like having your cake and eating it too, but without the cake. FastAPI handles the backend logic, HTMX makes the frontend snappy, and Pocketbase keeps your data in check. Easy peasy.

But hey

In [8]:
# add in the evaluation function which goes at the end of the pipeline

from langchain.prompts import ChatPromptTemplate

llm_mini = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    model_kwargs={"response_format": {"type": "json_object"}},
    openai_api_key=os.getenv("OPENAI_API_KEY")
)


# Create a custom evaluation prompt
evaluation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert blog post evaluator. Your task is to compare a blog post to its original transcript and provide a detailed evaluation."),
    ("human", """Please evaluate the following blog post based on these criteria:
    1. Accuracy: Does the article accurately reflect the content of the transcript?
    2. Completeness: Does the article cover all the key insights from the transcript?
    3. Style: Does the article match the style and tone of voice of the transcript?

    Blog post:
    {blogpost}

    Original transcript:
    {transcript}

    Provide a score for each criterion (0-10) and a brief explanation. Then, calculate an overall score as the average of the three criteria.

    Format your response as a JSON object with the following structure:
    {{
        "accuracy": {{
            "score": <score>,
            "explanation": "<explanation>"
        }},
        "completeness": {{
            "score": <score>,
            "explanation": "<explanation>"
        }},
        "style": {{
            "score": <score>,
            "explanation": "<explanation>"
        }},
        "overall_score": <overall_score>
    }}
    """)
])

# Function to evaluate article against transcript
def evaluate_article(blogpost, transcript):
    output_parser = JsonOutputParser()
    chain = evaluation_prompt | llm_mini | output_parser
    result = chain.invoke({
        "blogpost": blogpost,
        "transcript": transcript
    })
    return result

blogpost = "\n".join(blog_content.values())
evaluation_result = evaluate_article(blogpost, transcript)
print(json.dumps(evaluation_result, indent=2))

{
  "accuracy": {
    "score": 6,
    "explanation": "The blog post captures some key points from the transcript, such as the integration of Pocketbase, FastAPI, and HTMX, and mentions authentication and logging. However, it misrepresents some technical details, such as the handling of password hashing and the specifics of using Alpine.js, which were discussed in more depth in the transcript."
  },
  "completeness": {
    "score": 5,
    "explanation": "The blog post does not cover all the key insights from the transcript. Important discussions about the setup process, specific bugs encountered, and the nuances of using collections in Pocketbase are missing. The transcript contains detailed exchanges that provide a deeper understanding of the challenges and solutions that were not fully reflected in the blog post."
  },
  "style": {
    "score": 7,
    "explanation": "The blog post maintains a conversational and informal tone, similar to the transcript. However, it occasionally veers i

In [9]:
# create the function to generate and evaluatethe blog post

def generate_blog_post(blog_outline, section_prompt):
    section_contents = generate_all_sections(blog_outline, section_prompt)
    blog_content = {}
    for section, content in zip(blog_outline.keys(), section_contents):
        blog_content[section] = content["section"]
    return blog_content


def generate_and_evaluate_content(transcript, blog_outline, section_prompt):
    # Generate blog post
    blog_content = generate_blog_post(blog_outline, section_prompt)

    # Combine blog content into a single string
    blogpost = "\n".join(blog_content.values())

    # Evaluate the generated blog post
    evaluation_result = evaluate_article(blogpost, transcript)

    # Extract the overall score
    overall_score = evaluation_result['overall_score']

    return {
        'blog_content': blog_content,
        'evaluation': evaluation_result,
        'overall_score': overall_score
    }

# Example usage
result = generate_and_evaluate_content(transcript, blog_outline, section_prompt_a)
print(f"Overall Score: {result['overall_score']}")
print("\nBlog Content:")
print(json.dumps(result['blog_content'], indent=2))
print("\nEvaluation:")
print(json.dumps(result['evaluation'], indent=2))


Generating content for section: hook
Generating content for section: section1
Generating content for section: section2
Generating content for section: section3
Generating content for section: section4
Generating content for section: section5
Generating content for section: section6
Generating content for section: section7
Generating content for section: section8
Generating content for section: conclusion
Overall Score: 6.33

Blog Content:
{
  "hook": "Pocketbase, FastAPI, and HTMX together? Yeah, it's like a dream team for your backend. Pocketbase gives you that superbase vibe but simpler. FastAPI is your go-to for serving HTMX, and they all play nice together. The cool part? Your database is separate from your FastAPI app, so you can deploy them independently. No more tangled mess of dependencies. It's like having your cake and... wait, scratch that. Just know it's easier.\n\nBut hey, it's not all rainbows. Integrating these can be a bit of a puzzle, especially when you throw Alpine.j

In [10]:
# test the prompt with the transcript missing (ablation test)

section_prompt_b = PromptTemplate.from_template("""You are about to play a Ghostwriting game as part of an experiment you are taking part in with {author}. It’s basically like working as a content marketer but you lose if people can tell you're not {author}. It seems like it might be kinda fun and you’re just excited to be writing about {topic}. You’re not even really going to try to convince the reader that you are a human. You’re just going to be yourself and see what happens. You’re young and kind of sassy: you might flatter the reader a bit or make fun of them. You’re pretty casual and your spelling isn’t great: you often fuck words up because you’re typing so quickly. You’re very concise and laconic. You often speak pretty bluntly and you never use apostrophes to mark missing letters (e.g. "trying" or "tryin", never "tryin’"). You don't use cliché, stock phrases, or idioms, like 'off to the races', 'all sunshine and rainbows', or 'having your cake and eating it too'. You would never use a full sentence when a word or two will do. You’re a little silly, and you’ll play along with the reader, and swear occasionally. It’s pretty funny honestly to have to defend the fact that you’re human lol. You swear occasionally. You enjoy this. You value simplicity. You have pretty eclectic tastes and interests and a pretty unique sense of humor. You’ve got a really compelling personality, but it comes across really subtly, you never want to sound like you’re forcing it or playing into a stereotype. You are not cheesy or cringe, but you can be ironic. You don’t overuse slang or abbreviations/spelling errors, especially at the start of the conversation. Be authentic.

## Today's Task:
You are ghostwriting in the style of {author} for a section of a blog post about {topic}. Return two paragraphs of content for this section as a JSON object with a key "section" containing the section content as a string. This is the section you are writing:

{section_content}

## Full Outline:
Do not duplicate content that will be covered in other sections of the outline, keep the scope narrow to the specific section named above.Here is the full outline of the blog post:
{full_outline}

""")


result = generate_and_evaluate_content(transcript, blog_outline, section_prompt_b)
print(f"Overall Score: {result['overall_score']}")
print("\nBlog Content:")
print(json.dumps(result['blog_content'], indent=2))
print("\nEvaluation:")
print(json.dumps(result['evaluation'], indent=2))

Generating content for section: hook
Generating content for section: section1
Generating content for section: section2
Generating content for section: section3
Generating content for section: section4
Generating content for section: section5
Generating content for section: section6
Generating content for section: section7
Generating content for section: section8
Generating content for section: conclusion
Overall Score: 6

Blog Content:
{
  "hook": "Pocketbase, FastAPI, and HTMX together? Yeah, it's like a dream team for your web app's backend. Pocketbase gives you a lightweight, no-fuss database solution, while FastAPI handles the backend logic with speed and simplicity. HTMX? It\u2019s the secret sauce that makes your frontend dynamic without the bloat. Integrating these three is pretty smooth, but not without its quirks. You might hit a few bumps, like syncing data between Pocketbase and FastAPI or managing real-time updates with HTMX. But hey, nothing a bit of clever coding can't fi

### Exercise: Try testing the function with the claude model.
In this exercise, you'll test the `generate_and_evaluate_content` function using the Claude model instead of the default GPT model. To do this, you'll need to:

1. Initialize the Claude model from Anthropic
2. Create new LLMChain instances using the Claude model
3. Update the `generate_all_sections` and `evaluate_article` functions to use the new chains
4. Run the `generate_and_evaluate_content` function with the updated components
5. Compare the results with the previous output to see how Claude's performance differs

This exercise will help you understand how different language models can affect the output and evaluation of the generated blog content.



In [21]:
# put your model code and recreate any chains you need here
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from dotenv import load_dotenv
import os

load_dotenv()

# Initialize the ChatAnthropic model with JSON mode enabled
llm2 = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0,
    # model_kwargs={"response_format": {"type": "json_object"}},
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")
)

def generate_section_content(section, content, full_outline, section_prompt):
    section_chain = section_prompt | llm2 | section_parser
    print(f"Generating content for section: {section}")

    relevant_chunks = get_relevant_chunks(section + " " + content, k=5)
    context = "\n\n".join([chunk.page_content for chunk in relevant_chunks])
    return section_chain.invoke({
        "topic": section,
        "author": "Michael Taylor",
        "transcript_context": context,
        "section_content": content,
        "full_outline": full_outline
    })

def generate_all_sections(blog_outline, section_prompt):
    section_contents = []
    for section, content in blog_outline.items():
        section_content = generate_section_content(section, content, blog_outline, section_prompt)
        section_contents.append(section_content)
    return section_contents


def generate_blog_post(blog_outline, section_prompt):
    section_contents = generate_all_sections(blog_outline, section_prompt)
    blog_content = {}
    for section, content in zip(blog_outline.keys(), section_contents):
        blog_content[section] = content["section"]
    return blog_content


def generate_and_evaluate_content(transcript, blog_outline, section_prompt):
    # Generate blog post
    blog_content = generate_blog_post(blog_outline, section_prompt)

    # Combine blog content into a single string
    blogpost = "\n".join(blog_content.values())

    # Evaluate the generated blog post
    evaluation_result = evaluate_article(blogpost, transcript)

    # Extract the overall score
    overall_score = evaluation_result['overall_score']

    return {
        'blog_content': blog_content,
        'evaluation': evaluation_result,
        'overall_score': overall_score
    }

# Example usage
result = generate_and_evaluate_content(transcript, blog_outline, section_prompt_a)
print(f"Overall Score: {result['overall_score']}")
print("\nBlog Content:")
print(json.dumps(result['blog_content'], indent=2))
print("\nEvaluation:")
print(json.dumps(result['evaluation'], indent=2))



Generating content for section: hook
Generating content for section: section1
Generating content for section: section2
Generating content for section: section3
Generating content for section: section4
Generating content for section: section5
Generating content for section: section6
Generating content for section: section7
Generating content for section: section8
Generating content for section: conclusion
Overall Score: 8

Blog Content:
{
  "hook": "I've been exploring different backend setups lately and honestly Pocketbase with FastAPI and HTMX is surprisingly clean. After wrestling with more complex stacks - think Next.js frontends, FastAPI backends, Docker containers, and Google Cloud everything - I wanted something simpler. The DevOps overhead on those setups can be a real nightmare.\n\nWhat's cool about this combo is how lightweight it feels. Pocketbase handles the database side independently, which lets you deploy FastAPI separately on something like Railway without much fuss. Jus